# Find maximum drop in number density along vector(s) connecting nearest neighbour sites 

In [1]:
!pip install mp_api -q
!pip install mp-pyrho -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.1.0 requires pydantic!=1.8,!=1.8.1,<1.10.0,>=1.7.4, but you have pydantic 2.6.4 which is incompatible.
spacy 3.4.0 requires pydantic!=1.8,!=1.8.1,<1.10.0,>=1.7.4, but you have pydantic 2.6.4 which is incompatible.
gradient-utils 0.5.0 requires pymongo<4.0.0,>=3.11.0, but you have pymongo 4.6.2 which is incompatible.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import sys
sys.path.append('/notebooks/ranker/src/')
from get_from_mp_api import *

from pyrho.pgrid import PGrid
from pyrho.vis.scatter import get_scatter_plot
import pyrho.vis.scatter

No module named 'phonopy'
No module named 'phonopy'


In [3]:
def convert_to_list(string):
    if string.startswith('[') and string.endswith(']'):
        return eval(string)
    else:
        return string
def convert_string_to_dict(string_dict):
    return eval(string_dict)
main_df = pd.read_csv("../../Create_dataframe/df_full_flat_mat_info.csv", converters={'paper_titles': convert_to_list, "sub_lattice_structure": convert_string_to_dict})
main_df.set_index("material_id", inplace=True)
main_df['sub_lattice_structure'][0]

{'@module': 'pymatgen.core.structure',
 '@class': 'Structure',
 'charge': 0,
 'lattice': {'matrix': [[5.83749204, 0.06254491, -2.71890814],
   [-4.28350897, 3.9662647, -2.71900595],
   [0.02407769, 0.0625842, 6.43958097]],
  'pbc': (True, True, True),
  'a': 6.439929087854037,
  'b': 6.439929978087647,
  'c': 6.43993009173481,
  'alpha': 114.75185786877043,
  'beta': 114.75184782858017,
  'gamma': 114.75183455160276,
  'volume': 152.79889506945906},
 'properties': {},
 'sites': [{'species': [{'element': 'S', 'occu': 1}],
   'abc': [0.24492283, 0.14077002, 0.90933939],
   'xyz': [0.848640219099403, 0.6305601157552272, 4.807087432994953],
   'properties': {'magmom': -0.0},
   'label': 'S'},
  {'species': [{'element': 'S', 'occu': 1}],
   'abc': [0.90933939, 0.24492283, 0.14077002],
   'xyz': [4.262521728423524, 1.0371133042457898, -2.231856959615954],
   'properties': {'magmom': -0.0},
   'label': 'S'},
  {'species': [{'element': 'S', 'occu': 1}],
   'abc': [0.14077002, 0.90933939, 0.244

In [6]:
def get_nn_site(str_dict):
    # return positions of first and second sites (I believe they are in order of closest neighbours already 
    #but not certain)
    # will update this function to make more sophisticated search over neighbours if doesn't run too slow
    
    # if only a single atomic site present, return the corner of unit cell as comparison position
    if len(str_dict['sites'])==1:
        return [str_dict['sites'][0]['abc'], [1,1,1]]
    
    return [str_dict['sites'][0]['abc'], str_dict['sites'][1]['abc']]

# Define a function to return drop in number density between the two sites
def get_nn_density_drop(row):
    str_dict = row['sub_lattice_structure']
    mat_ID = row.name
    
    try:
        with MPRester(API_KEY) as mpr:
            charge_density = mpr.get_charge_density_from_material_id(mat_ID)
    except:
        return np.nan
    
    nn_positions = get_nn_site(str_dict)
    
    # sometimes one of the positions is very very slightly negative (leading to out of bounds error in interpolate)
    # when I believe it should be basically zero
    try:
        chg_den_interpolated = charge_density.linear_slice(nn_positions[0], nn_positions[1], n=70)
    except ValueError:
        for i in range(len(nn_positions)):
            if nn_positions[i]>1:
                nn_positions[i]=1
        chg_den_interpolated = charge_density.linear_slice(np.abs(np.array(nn_positions[0])), np.abs(np.array(nn_positions[1])), n=70)
        
        
    peak = max(chg_den_interpolated)

    return (peak-min(chg_den_interpolated))/peak

In [7]:
#print(main_df.loc["mp-7"].name)
# test for one mat
get_nn_density_drop(main_df.loc["mp-7"])

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving TaskDoc documents:   0%|          | 0/4 [00:00<?, ?it/s]

0.966837088871634

In [ ]:
# Apply the function to create a new column
MAX_MATERIAL_IDS_LENGTH = 1000
material_ids = main_df.index.to_list()

chunks = divide_chunks(material_ids, MAX_MATERIAL_IDS_LENGTH)

# Initialize an empty DataFrame to store drop values
drop_values_df = pd.DataFrame()

for chunk in chunks:
    drop_values = np.ones(len(chunk))*-1
    for i, ID in enumerate(chunk):
        drop_values[i] = get_nn_density_drop(main_df.loc[ID])
    drop_values_df = drop_values_df.append(pd.DataFrame({'Drop Values': drop_values}))

    # Save drop values to CSV file
    drop_values_df.to_csv('drop_values.csv', index=False)

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving TaskDoc documents:   0%|          | 0/4 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving TaskDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

...

Retrieving TaskDoc documents:   0%|          | 0/11 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving TaskDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving TaskDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
#print(type(den.structure.lattice.matrix))
print(type(den))
print(den.structure.lattice)
#print(den.data)
len(den.data["total"])

NameError: name 'den' is not defined

In [4]:
den.

Type:           Chgcar
String form:    <pymatgen.io.vasp.outputs.Chgcar object at 0x7f81b90b6a00>
File:           /usr/local/lib/python3.9/dist-packages/pymatgen/io/vasp/outputs.py
Source:        
class Chgcar(VolumetricData):
    """Simple object for reading a CHGCAR file."""

    def __init__(self, poscar, data, data_aug=None):
        """
        Args:
            poscar (Poscar | Structure): Object containing structure.
            data: Actual data.
            data_aug: Augmentation charge data.
        """
        # allow for poscar or structure files to be passed
        if isinstance(poscar, Poscar):
            struct = poscar.structure
            self.poscar = poscar
            self.name = poscar.comment
        elif isinstance(poscar, Structure):
            struct = poscar
            self.poscar = Poscar(poscar)
            self.name = None

        super().__init__(struct, data, data_aug=data_aug)
        self._distance_matrix = {}

    @classmethod
    def from_file(c

In [40]:
den.make_supercell??

Object `den.make_supercell` not found.


In [45]:
den.reverse??

Object `den.reverse` not found.


In [66]:
den.structure.substitute??

Signature:
den.structure.substitute(
    index: 'int',
    func_group: 'IMolecule | Molecule | str',
    bond_order: 'int' = 1,
) -> 'None'
Source:   
    def substitute(self, index: int, func_group: IMolecule | Molecule | str, bond_order: int = 1) -> None:
        """Substitute atom at index with a functional group.

        Args:
            index (int): Index of atom to substitute.
            func_group: Substituent molecule. There are two options:

                1. Providing an actual Molecule as the input. The first atom
                   must be a DummySpecies X, indicating the position of
                   nearest neighbor. The second atom must be the next
                   nearest atom. For example, for a methyl group
                   substitution, func_group should be X-CH3, where X is the
                   first site and C is the second site. What the code will
                   do is to remove the index site, and connect the nearest
                   neighbor to t

In [74]:
den.structure.make_supercell((3, 3, 3))

Structure Summary
Lattice
    abc : 16.56160646213241 16.56160646213241 16.56160646213241
 angles : 98.30796159355172 98.30796159355172 135.31736305075154
 volume : 2954.5071546728477
      A : -10.831761 10.831761 6.295452
      B : 10.831761 -10.831761 6.295452
      C : 10.831761 10.831761 -6.295452
    pbc : True True True
PeriodicSite: Be (1.805, 1.805, 1.049) [0.1667, 0.1667, 0.1667]
PeriodicSite: Be (5.416, 5.416, -1.049) [0.1667, 0.1667, 0.5]
PeriodicSite: Be (9.026, 9.026, -3.148) [0.1667, 0.1667, 0.8333]
PeriodicSite: Be (5.416, -1.805, 3.148) [0.1667, 0.5, 0.1667]
PeriodicSite: Be (9.026, 1.805, 1.049) [0.1667, 0.5, 0.5]
PeriodicSite: Be (12.64, 5.416, -1.049) [0.1667, 0.5, 0.8333]
PeriodicSite: Be (9.026, -5.416, 5.246) [0.1667, 0.8333, 0.1667]
PeriodicSite: Be (12.64, -1.805, 3.148) [0.1667, 0.8333, 0.5]
PeriodicSite: Be (16.25, 1.805, 1.049) [0.1667, 0.8333, 0.8333]
PeriodicSite: Be (-1.805, 5.416, 3.148) [0.5, 0.1667, 0.1667]
PeriodicSite: Be (1.805, 9.026, 1.049) [0.5, 

In [76]:
den.structure

Structure Summary
Lattice
    abc : 16.56160646213241 16.56160646213241 16.56160646213241
 angles : 98.30796159355172 98.30796159355172 135.31736305075154
 volume : 2954.5071546728477
      A : -10.831761 10.831761 6.295452
      B : 10.831761 -10.831761 6.295452
      C : 10.831761 10.831761 -6.295452
    pbc : True True True
PeriodicSite: Be (1.805, 1.805, 1.049) [0.1667, 0.1667, 0.1667]
PeriodicSite: Be (5.416, 5.416, -1.049) [0.1667, 0.1667, 0.5]
PeriodicSite: Be (9.026, 9.026, -3.148) [0.1667, 0.1667, 0.8333]
PeriodicSite: Be (5.416, -1.805, 3.148) [0.1667, 0.5, 0.1667]
PeriodicSite: Be (9.026, 1.805, 1.049) [0.1667, 0.5, 0.5]
PeriodicSite: Be (12.64, 5.416, -1.049) [0.1667, 0.5, 0.8333]
PeriodicSite: Be (9.026, -5.416, 5.246) [0.1667, 0.8333, 0.1667]
PeriodicSite: Be (12.64, -1.805, 3.148) [0.1667, 0.8333, 0.5]
PeriodicSite: Be (16.25, 1.805, 1.049) [0.1667, 0.8333, 0.8333]
PeriodicSite: Be (-1.805, 5.416, 3.148) [0.5, 0.1667, 0.1667]
PeriodicSite: Be (1.805, 9.026, 1.049) [0.5, 

In [83]:
den.interpolator??

Signature:      den.interpolator(xi, method=None)
Type:           RegularGridInterpolator
String form:    <scipy.interpolate._rgi.RegularGridInterpolator object at 0x7f7041958190>
File:           /usr/local/lib/python3.9/dist-packages/scipy/interpolate/_rgi.py
Source:        
class RegularGridInterpolator:
    """
    Interpolator on a regular or rectilinear grid in arbitrary dimensions.

    The data must be defined on a rectilinear grid; that is, a rectangular
    grid with even or uneven spacing. Linear, nearest-neighbor, spline
    interpolations are supported. After setting up the interpolator object,
    the interpolation method may be chosen at each evaluation.

    Parameters
    ----------
    points : tuple of ndarray of float, with shapes (m1, ), ..., (mn, )
        The points defining the regular grid in n dimensions. The points in
        each dimension (i.e. every elements of the points tuple) must be
        strictly ascending or descending.

    values : array_like, sha

In [100]:
den.interpolator.__dir__()

['method',
 'bounds_error',
 'grid',
 '_descending_dimensions',
 'values',
 'fill_value',
 '__module__',
 '__doc__',
 '_SPLINE_DEGREE_MAP',
 '_SPLINE_METHODS',
 '_ALL_METHODS',
 '__init__',
 '_check_dimensionality',
 '_check_points',
 '_check_values',
 '_check_fill_value',
 '__call__',
 '_prepare_xi',
 '_evaluate_linear',
 '_evaluate_nearest',
 '_validate_grid_dimensions',
 '_evaluate_spline',
 '_do_spline_fit',
 '_do_pchip',
 '_find_indices',
 '_find_out_of_bounds',
 '__dict__',
 '__weakref__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__new__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [94]:
den.interpolator??

Signature:      den.interpolator(xi, method=None)
Type:           RegularGridInterpolator
String form:    <scipy.interpolate._rgi.RegularGridInterpolator object at 0x7f7041958190>
File:           /usr/local/lib/python3.9/dist-packages/scipy/interpolate/_rgi.py
Source:        
class RegularGridInterpolator:
    """
    Interpolator on a regular or rectilinear grid in arbitrary dimensions.

    The data must be defined on a rectilinear grid; that is, a rectangular
    grid with even or uneven spacing. Linear, nearest-neighbor, spline
    interpolations are supported. After setting up the interpolator object,
    the interpolation method may be chosen at each evaluation.

    Parameters
    ----------
    points : tuple of ndarray of float, with shapes (m1, ), ..., (mn, )
        The points defining the regular grid in n dimensions. The points in
        each dimension (i.e. every elements of the points tuple) must be
        strictly ascending or descending.

    values : array_like, sha

In [81]:
den.linear_slice??

Signature: den.linear_slice(p1, p2, n=100)
Source:   
    def linear_slice(self, p1, p2, n=100):
        """
        Get a linear slice of the volumetric data with n data points from
        point p1 to point p2, in the form of a list.

        Args:
            p1 (list): 3-element list containing fractional coordinates of the first point.
            p2 (list): 3-element list containing fractional coordinates of the second point.
            n (int): Number of data points to collect, defaults to 100.

        Returns:
            List of n data points (mostly interpolated) representing a linear slice of the
            data from point p1 to point p2.
        """
        assert type(p1) in [list, np.ndarray]
        assert type(p2) in [list, np.ndarray]
        assert len(p1) == 3
        assert len(p2) == 3
        xpts = np.linspace(p1[0], p2[0], num=n)
        ypts = np.linspace(p1[1], p2[1], num=n)
        zpts = np.linspace(p1[2], p2[2], num=n)
        return [self.value_at(xpts[

In [75]:
den.linear_slice([0, 0, 0], [5, 5, 5])

ValueError: One of the requested xi is out of bounds in dimension 0

In [64]:
den.structure.get_all_neighbors(2.6)

[[PeriodicNeighbor: Be (0.0, 2.532, 0.0) [0.3506, 0.0, 0.3506],
  PeriodicNeighbor: Be (0.0, 1.523, 2.098) [0.7109, 0.5, 0.2109],
  PeriodicNeighbor: Be (2.532, 0.0, 0.0) [0.0, 0.3506, 0.3506],
  PeriodicNeighbor: Be (1.523, 0.0, 2.098) [0.5, 0.7109, 0.2109],
  PeriodicNeighbor: Be (1.805, 1.805, -1.049) [0.0, 0.0, 0.5],
  PeriodicNeighbor: Be (1.805, 1.805, 3.148) [1.0, 1.0, 0.5],
  PeriodicNeighbor: Be (2.088, 3.611, 0.0) [0.5, 0.2891, 0.7891],
  PeriodicNeighbor: Be (1.079, 3.611, 2.098) [1.0, 0.6494, 0.6494],
  PeriodicNeighbor: Be (3.611, 2.088, 0.0) [0.2891, 0.5, 0.7891],
  PeriodicNeighbor: Be (3.611, 1.079, 2.098) [0.6494, 1.0, 0.6494]],
 [PeriodicNeighbor: Be (-2.532, 0.0, 0.0) [0.0, -0.3506, -0.3506],
  PeriodicNeighbor: Be (-3.611, 2.088, 0.0) [0.2891, -0.5, -0.2109],
  PeriodicNeighbor: Be (-3.611, 1.079, 2.098) [0.6494, 0.0, -0.3506],
  PeriodicNeighbor: Be (-2.088, 3.611, 5.551e-17) [0.5, -0.2891, 0.2109],
  PeriodicNeighbor: Be (-1.523, 0.0, 2.098) [0.5, 0.2891, -0.2109]

In [61]:
pymatgen.core.structure.PeriodicNeighbor??

Init signature:
pymatgen.core.structure.PeriodicNeighbor(
    species: 'Composition',
    coords: 'np.ndarray',
    lattice: 'Lattice',
    properties: 'dict | None' = None,
    nn_distance: 'float' = 0.0,
    index: 'int' = 0,
    image: 'tuple' = (0, 0, 0),
    label: 'str | None' = None,
) -> 'None'
Source:        
class PeriodicNeighbor(PeriodicSite):
    """Simple PeriodicSite subclass to contain a neighboring atom that skips all
    the unnecessary checks for speed. Can be used as a fixed-length tuple of
    size 4 to retain backwards compatibility with past use cases.

        (site, distance, index, image).

    In future, usage should be to call attributes, e.g., PeriodicNeighbor.index,
    PeriodicNeighbor.distance, etc.
    """

    def __init__(
        self,
        species: Composition,
        coords: np.ndarray,
        lattice: Lattice,
        properties: dict | None = None,
        nn_distance: float = 0.0,
        index: int = 0,
        image: tuple = (0, 0, 0),
  

In [15]:
# Extract charge density
charge_density = den.data['total']

# Determine atom indices corresponding to the element of interest
element_indices = [i for i, atom in enumerate(den.structure.species) if atom.symbol == 'S']

In [6]:
pg3d = PGrid(grid_data=den.data['total'], lattice=den.structure.lattice.matrix.tolist())

mpl.rc("image", cmap="viridis")
vis.scatter(pg3d.grid_data, pg3d.lattice, marker_size=1)

NameError: name 'vis' is not defined